In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

import tkinter as tk
from tkinter import filedialog

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.compat import lzip
from __future__ import print_function

In [ ]:
root = tk.Tk()
root.withdraw()
path = filedialog.askopenfilename()

In [ ]:
df = pd.read_pickle(path)

In [ ]:
df.columns

In [ ]:
result = df[(df["大通路"] == "BSSC") & (df["國別"] == "TH") & (df["規格"] == "315/80R22.5") & (df["花紋"] == "R150")]

In [ ]:
result["實際出貨日"] = pd.to_datetime(result["實際出貨日"], yearfirst=True)

In [ ]:
final = result.groupby([pd.Grouper(key="實際出貨日", freq="m")])[["銷售數量", "銷貨單價"]].agg({"銷售數量": "sum", "銷貨單價": "mean"})

In [ ]:
final.columns = ["Sales Volume", "AVG. unit price"]

In [ ]:
final.plot.line(subplots=True)

In [ ]:
final.plot.scatter(x="Sales Volume", y="AVG. unit price", c="g", s=18)

In [ ]:
# 獲取自變量和因變量
x = final["Sales Volume"]
y = final["AVG. unit price"]
X = sm.add_constant(x)

ols_model = sm.OLS(y, X)
answer = ols_model.fit()

print(answer.summary2())